# QuackOSM Command Line Interface

**QuackOSM** contains a CLI for users convenience. It is **not** installed by default when installed using `pip install quackosm`.

To include the CLI, **QuackOSM** has to be installed with additional group called `cli`: `pip install quackosm[cli]`.

CLI is based on the `typer` library and exposes almost all of the features implemented in the Python API.

After installation, the `QuackOSM` (or `quackosm`) command will be available in the shell.

Each command error returns a verbose description what went wrong.

In [ ]:
# Extend the default console width from 80 characters
import os

os.environ["COLUMNS"] = "160"

## Basic usage

By default, the quackosm requires just the path to the `PBF` file. Without it, there will be an error.

In [ ]:
! QuackOSM

Let's download a small extract and test the basic usage.

In [ ]:
! QuackOSM https://download.geofabrik.de/europe/andorra-latest.osm.pbf

Second execution of this command will immediately return a path to the previously generated file.

Since the file is already downloaded, we can use it directly.

In [ ]:
! QuackOSM files/andorra-latest.osm.pbf

To force the regeneration of the GeoParquet file, add the `--ignore-cache` flag (or `--no-cache`) to the command.

In [ ]:
! QuackOSM files/andorra-latest.osm.pbf --ignore-cache

## Help command

To get the full description of all arguments of the QuackOSM command, you can use the `--help` (or `-h`) parameter.

In [ ]:
! QuackOSM --help

## Geometry filters

QuackOSM can automatically download required PBF files based on multiple geometry filters:
- Text to geocode using Nominatim
- WKT geometry
- GeoJSON geometry
- Geometry file path
- H3 spatial index
- Geohash spatial index
- S2 spatial index

These filters can also be used to filter out geometries from provided pbf file.

`QuackOSM` will raise an error if provided geometry has parts without area (such as Points, LineStrings or empty geometry).

Let's see the example based on Monaco region.

First, we will visualise multiple filters on the map.

In [ ]:
import geopandas as gpd

from quackosm.cli import (
    GeocodeGeometryParser,
    GeohashGeometryParser,
    GeoJsonGeometryParser,
    H3GeometryParser,
    S2GeometryParser,
    WktGeometryParser,
)

In [ ]:
geocode_string = "Monaco-Ville, Monaco"
geojson_string = """{"type":"Feature","geometry":{"coordinates":[[[7.416,43.734],[7.416,43.731],[7.421,43.731],[7.421,43.734],[7.416,43.734]]],"type":"Polygon"}}"""
wkt_string = "POLYGON ((7.414 43.735, 7.414 43.732, 7.419 43.732, 7.419 43.735, 7.414 43.735))"
h3_string = "893969a4037ffff"
geohash_string = "spv2bcs"
s2_string = "12cdc28dc"

In [ ]:
geometry_types = ["Geocode", "GeoJSON", "WKT", "H3", "GeoHash", "S2"]
geometries = [
    GeocodeGeometryParser().convert(geocode_string),
    GeoJsonGeometryParser().convert(geojson_string),
    WktGeometryParser().convert(wkt_string),
    H3GeometryParser().convert(h3_string),
    GeohashGeometryParser().convert(geohash_string),
    S2GeometryParser().convert(s2_string),
]
gpd.GeoDataFrame(
    data=dict(type=geometry_types),
    geometry=geometries,
    crs=4326,
).explore(column="type", tiles="CartoDB positron")

Now we will execute each filter and let QuackOSM find required region on its own.

During first execution, QuackOSM will cache three PBF files sources locally. This operation takes some time.

The `--silent` flag will disable the progress output to the terminal.

### Geocoding

In [ ]:
! QuackOSM --geom-filter-geocode 'Monaco-Ville, Monaco' --silent --output files/geocode_example.parquet

### GeoJSON

In [ ]:
! QuackOSM --geom-filter-geojson '{"type":"Feature","geometry":{"coordinates":[[[7.416,43.734],[7.416,43.731],[7.421,43.731],[7.421,43.734],[7.416,43.734]]],"type":"Polygon"}}' --silent --output files/geojson_example.parquet

### Geohash

In [ ]:
! QuackOSM --geom-filter-index-geohash spv2bcs --silent --output files/geohash_example.parquet

### H3

In [ ]:
! QuackOSM --geom-filter-index-h3 893969a4037ffff --silent --output files/h3_example.parquet

### S2

In [ ]:
! QuackOSM --geom-filter-index-s2 12cdc28dc --silent --output files/s2_example.parquet

### WKT

In [ ]:
! QuackOSM --geom-filter-wkt 'POLYGON ((7.414 43.735, 7.414 43.732, 7.419 43.732, 7.419 43.735, 7.414 43.735))' --silent --output files/wkt_example.parquet

Plot all results for comparison

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(2, 3, sharex=True, sharey=True, figsize=(10, 6))

for idx, (geometry_type, geometry) in enumerate(zip(geometry_types, geometries)):
    ax = axs[idx // 3, idx % 3]
    gdf = gpd.read_parquet(f"files/{geometry_type.lower()}_example.parquet")
    gdf.plot(ax=ax, markersize=1, zorder=1, alpha=0.8)
    gdf.boundary.plot(ax=ax, markersize=0, zorder=1, alpha=0.8)
    gpd.GeoSeries([geometry], crs=4326).plot(
        ax=ax,
        color=(0, 0, 0, 0),
        zorder=2,
        hatch="///",
        edgecolor="orange",
        linewidth=1.5,
    )
    ax.set_title(geometry_type)

fig.tight_layout()

## OSM tags filters

By default, QuackOSM parses all of the features (nodes, ways, relations) from the `*.osm.pbf` file with tags attached.

Hovewer, there is also an option to pass an OSM tags filter in the form of JSON string or path to the JSON file.

OSM tags filter logic is based on the filter from the [`OSMnx`](https://github.com/gboeing/osmnx) library.

Filter is expected to be in the form of dictionary with `keys` as string and `values` as one of the types: string, list of strings or bool value. Full tutorial for OSM tags filters can be accessed [here](../advanced_examples/osm_tags_filter).

Example filters:

- All of the buildings
  ```json
  { "building": true }
  ```
- Parkings and offices
  ```json
  {
    "amenity": "parking",
    "building": "office"
  }
  ```
- General shops
  ```json
  {
    "shop": [
      "convenience",
      "department_store",
      "general",
      "kiosk",
      "mall",
      "supermarket",
      "wholesale"
    ]
  }
  ```

Tags filters can be used together with geometry filters to get specific features from the area of interest.

<div class="admonition info">
    <p class="admonition-title">Info</p>
    <p>
    By default, without any tags filters, QuackOSM returns all of the tags of the feature grouped as a single column: <strong>tags</strong>.<br />
    With tags filter, result file will keep the used in the filter and keep each tag key as a separate column.<br />
    To keep all tags while filtering the file, use <strong>--keep-all-tags</strong> flag.
    </p>
</div>

In [ ]:
! QuackOSM files/andorra.osm.pbf --osm-tags-filter '{ "building": true }'

## Keeping tags compact or separate

QuackOSM can keep tags in the compact form (as a single column named `tags`) or wide form (each tag key as separate column).
If not set by the user, it will change depending on the presence of tags filter:
- without tags filter: tags kept together as a sinlge column
- with tags filter: tags split into separate columns

User can force one of those two behaviours regardless of osm tags filter being present or not:
- `--explode-tags` (or `--explode`): will always split tags into separate columns, sometimes resulting in hundreds or event thousands of columns in the result file.
- `--compact-tags` (or `--compact`): will always keep tags together as a single column.

### Separated tags (`explode`)

In [ ]:
! QuackOSM files/andorra.osm.pbf --osm-tags-filter '{ "amenity": "parking", "building": "office" }' --explode --output files/andorra_filtered_exploded.geoparquet --silent

In [ ]:
! ./duckdb :memory: "FROM read_parquet('files/andorra_filtered_exploded.parquet')"

### Compact tags (`compact`)

In [ ]:
! QuackOSM files/andorra.osm.pbf --osm-tags-filter '{ "amenity": "parking", "building": "office" }' --compact --output files/andorra_filtered_compact.geoparquet --silent

In [ ]:
! ./duckdb :memory: "FROM read_parquet('files/andorra_filtered_compact.parquet')"

## WKT mode

By default, QuackOSM saves parsed files in the `GeoParquet` format with the geometry in the `WKB` format.

There is also an option to save the file as a `Parquet` file with the geometry in the `WKT` format using `--wkt-result` (or `--wkt`) parameter.

In [ ]:
! QuackOSM files/andorra.osm.pbf --wkt-result --silent

In [ ]:
! ./duckdb :memory: "FROM read_parquet('files/andorra_nofilter_noclip_compact_wkt.parquet')"